<h1><center>Analyse der Word Embedding Modelle des Neuen Pitaval (1842-1890) </center></h1>

Das Jupyter Notebook führt Analysen mit vortrainierten Word Embedding Modellen des *Neuen Pitaval* durch und erstellt anschließend Abbildung 8 (sowie Anhang 3 und 4).

### Ordnerstruktur
####      modelle (enthält bereits erstellte Modelle)
####      results (werden automatisch erstellt)
    

## 0. Laden der benötigten Bibliotheken

In [ ]:
# Word Embedding
from gensim.models import word2vec
import pickle

# Visualisierung
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np

import os


In [ ]:
# Dateipfade anpassen
path_modelle = r"...\modelle"
path_results = r"...\results"

os.chdir(path_modelle)

## 1. Einlesen der Modelle und Daten

Für die Analysen des *Neuen Pitaval* wurden insgesamt drei Word Embedding Modelle erstellt. Ein Modell für den gesamten Zeitraum (1842-1890) und jeweils ein Modell für den Zeitraum der Herausgeberschaft unter Julius Eduard Hitzig und Wilhelm Häring (Willibald Alexis) (Band 1-30 bzw. 1842-1862) sowie unter Anton Vollert (Band 31-60 bzw. 1862-1890). Das gesamte Korpus umfasst (nach dem Preprocessing) 6.720.771 Token, die einzelnen Teilkorpora jeweils 3.577.314 und 3.143.457 Token.

Die Word Embedding Modelle basieren auf Word2Vec (Mikolov et al. 2013) und dessen Implementierung im Python-Package *gensim*. Im Preprocessing wurden Satzzeichen entfernt, alle Wörter lemmatisiert (mit dem Package *spacy*) und kleingeschrieben. Die Modelle wurden mit den folgenden Parametern erstellt: Größe der Vektoren = 300, Größe des Kontextfensters = 5, mind. Häufigkeit der Zielwörter = 10, Architektur = Skipgram, Iterationen = 50. Daraus resultieren ein gesamtes Modell zu 22.805 Types sowie die Teilkorpora zu 14.922 Types (Hitzig/Häring) und 14.506 Types (Vollert).

Die Analysen im Artikel beziehen sich auf das Gesamtkorpus (*modelPitaval*), die Modelle der Teilkorpora können für vergleichende und weiterführende Analysen genutzt werden.

In [ ]:
# Einlesen der Modelle

modelPitaval = word2vec.KeyedVectors.load('modelPitaval_vectors.kv')

In [ ]:
modelHaering = word2vec.KeyedVectors.load('modelHaering_vectors.kv')

modelVollert = word2vec.KeyedVectors.load('modelVollert_vectors.kv')

In [ ]:
# Einlesen der Korpora (in ihrer Version nach dem Preprocessing)

with open('corpusPitaval_words.pkl', 'rb') as f:
    corpusPitaval = pickle.load(f)

In [ ]:
with open('corpusHaering_words.pkl', 'rb') as f:
    corpusHaering = pickle.load(f)

In [ ]:
with open('corpusVollert_words.pkl', 'rb') as f:
    corpusVollert = pickle.load(f)

In [ ]:
# Informationen zu Modellen und Korpora anzeigen

print("Größe des Modells: " + str(len(modelPitaval)) + " Types, Größe des Korpus: " + str(len(corpusPitaval)) + " Tokens")

In [ ]:
print("Größe des Modells: " + str(len(modelHaering)) + " Types, Größe des Korpus: " + str(len(corpusHaering)) + " Tokens")

In [ ]:
print("Größe des Modells: " + str(len(modelVollert)) + " Types, Größe des Korpus: " + str(len(corpusVollert)) + " Tokens")

## 2. Visualisierungen einer Auswahl von Wörtern auf einer bestimmten Ebene

Der nachfolgende Code stellt dar, wie eine Gruppe von Straftaten sich in ihrer Wertigkeit auf durch Wortpaare aufgespannte Achsen verhält. Die x-Achse wird dabei immer von dem paar "Verbrechen"<->"Vergehen" bestimmt. Die y-Achse wird variiert.
Folgende Paare werden untersucht: "Verbrecher" <-> "Verbrecherin", "gut" <-> "böse", "Mann" <-> "Frau", "Herr" <-> "Dame", "Thäter" <-> "Thäterin" 

Quelle: Gergely Nemeth, https://colab.research.google.com/drive/1TCgnpIwsr6uK4cP0Gk1RCCQrFy6s3Xc1

In [ ]:
# Ergebnisordner erstellen
path_results = path_results + r"\WordEmbeddings"

if os.path.exists(path_results) == False:
    os.mkdir(path_results)

os.chdir(path_results)

In [ ]:
# Liste der Wortpaare, die die y-Achse aufspannen
wordpairs = [("verbrecher", "verbrecherin"), ("böse", "gut"), ("mann", "frau"), ("herr", "dame"), ("thäter", "thäterin"), ("gut", "böse")]

In [ ]:
# Bestimmen des zu untersuchenden Wortpaares (0: "verbrecher" <-> "verbrecherin", 1: "böse" <-> "gut", 2: "mann" <-> "frau", 3: "herr" <-> "dame", 4: "thäter" <-> "thäterin", 5: "gut" <-> "böse")
n = 0  
# n = 1
# n = 2
# n = 3
# n = 4
# n = 5

In [ ]:
# Wortlistenerstellung (W1), die ersten vier Wörter beschreiben die Achsen, die nachfolgenden die Zielwörter
Wl = ['verbrechen', 'vergehen', wordpairs[n][1], wordpairs[n][0], 'mord', 'brandstiftung', 'diebstahl', 'einbruch', 'betrug', 'veruntreuung', 'fälschung', 'giftmord']

In [ ]:
# Die Vektoren (Embeddings) jedes Wortes aus der Liste W1 werden in einer weiteren Liste Wv abgespeichert 
Wv = []
for word in Wl:
    Wv.append(modelPitaval.get_vector(word))

In [ ]:
# Berechnen der Differenzvektoren zwischen den Embeddings der jeweiligen Wörter, die die Achsen bilden sollen
# Daraus wird die neue Basis gebildet
b1 = (Wv[1]-Wv[0])
b2 = (Wv[3]-Wv[2])

In [ ]:
# Wir wollen die Vektoren auf einen zweidimensionalen Vektorraum mit Basis (1,0) und (0,1) projezieren
# Da B.T (1,0) = b1 und B.T (0,1) = b2, müssen wir wir für jeden Vektor v in Wv folgendes Gleichungssystem lösen: B.T x = v
# Da B.T nicht quadratisch und nicht invertierbar ist, nähern wir die Lösung mit einer Pseudoinversen Matrix an
#  
#
# Konvertieren der Liste der Wortvektoren in ein numpy-Array
W = np.array(Wv)
# Erstellen einer Matrix B aus den Basisvektoren b1 und b2
B = np.array([b1,b2])
# Berechnen der Pseudoinversen der Transponierten der Matrix B
Bi = np.linalg.pinv(B.T)

In [ ]:
# Multiplizieren der Pseudoinversen Bi mit der Matrix, die die Wortvektoren enthält 
# Wp ist eine 2xn-Matrix die alle projezierten, zweidimensionalen Wortvektoren enthält
Wp = np.matmul(Bi,W.T)
# Wir verschieben die definierten Achsen auf die x- und y-Achse
# Dafür subtrahieren wir komponentenweise auf der x-Achse den x-Wert der aufgespannten vertikalen Achse (Wp[0,2]), sowie auf der y-Achse den y-Wert der aufgespannten waagerechten Achse (Wp.[1,0]) 
Wp = (Wp.T-[Wp[0,2],Wp[1,0]]).T

In [ ]:
# Grafik: Scatter-Plot, Visualisierung der projezierten Vektoren
# Jeder Punkt ist beschriftet und stellt ein Wort aus der Liste dar
plt.figure(figsize=(12,7))
plt.axvline()
plt.axhline()
plt.scatter(Wp[0,:], Wp[1,:])
rX = max(Wp[0,:])-min(Wp[0,:])
rY = max(Wp[1,:])-min(Wp[1,:])
eps = 0.005
for i, txt in enumerate(Wl):
    plt.annotate(txt, (Wp[0,i]+rX*eps, Wp[1,i]+rX*eps))
plt.show()

plt.savefig("sem_scale_" + W1[3] + "-" + W1[2] + ".png", format='png', dpi=150, bbox_inches='tight')